# GRT Class Design

Data structure:

- **Body**. Properties and methods of a body.
    - Depends on: SPICE

- **Observer**. Describes the properties and provides methods of a geographical location on a body.
    - Depends on: SPICE

- **Gravitational field**. Properties and methods of the gravitational field where the rays are thrown.
    - Depends on SPICE

- **Ray**. Describes the properties and provides the methods of a given GRT Ray.

- **NEODistribution**. Describes the properties and provides the method of a theoretical NEOs 
    distribution using multidimensional gaussians.

- **Conic**. Properties and methods of a Keplerian orbit.

- **PatchedConic**. Properties and methods of a patched conic.

Examples:

```python
#Objects involved
moon=Body("MOON","IAU_MOON")
earth=Body("EARTH","ITRF93")
ssb=Body("SSB",None)

#Scenario (location in space-time and grav. fields where the ray is propagated
scenario=Scenario(moon,[earth,ssb])

#Gravitational field and location
crater=Location(scenario,latitude,longitude,altitude)

#Ray
tdb=spy.str2t("02/15/2013 03:20:34 UTC")
ray=GrtRay(crater,azimuth,elevation,speed)
ray.location.updateLocation(tdb)
ray.propagateRay()

#Terminal orbit
orbit=ray.getTerminalOrbit()
jacobian=ray.getTerminalJacobian()
print("Orbital elements:",orbit.elements)
print("Unbound orbital elements:",orbit.unelements)
print("State vector:",orbit.state)
print("Jacobian:\n",jacobian)

#Compute ray probability
dist=NEODistribution(parameters)
pelements=dist.calcDensity(orbit.unelements)
pray=ray.calcDensity(pelements)

#Same location multiple times
for t in np.linspace(tdb,tdb+30*DAYS):
    ray.location.updateLocation(t)
    ray.propagateRay()
    p=ray.calcProbability(dist)
    
#Same time multiple locations
craters=[
    Location(scenario,latitude1,longitude1,altitude1),
    Location(scenario,latitude2,longitude2,altitude2)
]
tdb=spy.str2t("02/15/2013 03:20:34 UTC")
for crater in craters:
    crater.updateLocation(tdb)
for crater in craters:
    ray=GrtRay(crater,azimuth,elevation,speed)
    ray.propagateRay()
    p=ray.calcProbability(dist)
```

In [1]:
import spiceypy as spy
import numpy as np
import unittest
from copy import deepcopy

In [277]:
class Util(object):
    
    def fin2Inf(x,scale=1):
        """
        Map variable x from the interval [0,scale] to a new variable t in the interval [-inf,+inf].
        x = 0 correspond to t->-inf
        x = 1 correspond to t->+inf
        """
        u=x/scale
        t=np.log(u/(1-u))
        return t

    def inf2Fin(t,scale=1):
        """
        Map variable t from the interval (-inf,inf) to a new variable x in the interval [0,scale].
        t->-inf correspond to x = 0
        t->+inf correspond to x = 1
        """
        x=scale/(1+np.exp(-t))
        return x
    
    def genIndex(probs):
        """
        Given a set of (normalized) randomly generate the index n following the probabilities.
        For instance if we have 3 events with probabilities 0.1, 0.7, 0.2, genSample will generate
        a number in the set (0,1,2) with those probabilities.
        
        Parameters:
            probs: Probabilities, numpy array (N), adimensional
            NOTE: It should be normalized, ie. sum(probs)=1
            
        Return:
            n: Index [0,1,2,... len(probs)-1], integer
        """
        cond=(np.random.rand()-np.cumsum(probs))<0
        isort=np.arange(len(probs))
        n=isort[cond][0] if sum(cond)>0 else isort[0]
        return n
    
#Unitary test
class Test(unittest.TestCase):

    probs=np.array([0.1,0.2,0.40,0.3])

    #"""
    def test_gen_index(self):
        N=10000
        M=len(probs)
        ns=np.array([Util.genIndex(self.probs) for i in range(N)]+[M])
        h,nx=np.histogram(ns,4)
        print(h/N)
        self.assertEqual(np.isclose(h/N,
                                    self.probs,
                                    rtol=1e-1).tolist(),
                         [True]*M)

    def test_fin2inf(self):
        self.assertAlmostEqual(Util.fin2Inf(0.001,scale=1),-6.906754778648554,13)
        self.assertAlmostEqual(Util.fin2Inf(0.78,scale=1),1.265666373331276,13)
        self.assertAlmostEqual(Util.fin2Inf(np.pi,scale=2*np.pi),0.0,13)

    def test_inf2fin(self):
        self.assertAlmostEqual(Util.inf2Fin(-3.0,scale=1),0.04742587317756678,13)
        self.assertAlmostEqual(Util.inf2Fin(+3.0,scale=1),0.9525741268224334,13)
        self.assertAlmostEqual(Util.inf2Fin(+3.0,scale=10.0),9.525741268224333,13)

    #""" 
    
    def timing_fin2inf(self):
        Util.fin2Inf(0.78,scale=1)
    def timing_inf2fin(self):
        Util.inf2Fin(-5.4,scale=1)
    def timing_gen_index(self):
        Util.genIndex(probs)
    
if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    #"""
    print("Timing fin2inf:")
    %timeit -n 1000 Test().timing_fin2inf()

    print("Timing inf2fin:")
    %timeit -n 1000 Test().timing_inf2fin()

    print("Timing genindex:")
    %timeit -n 1000 Test().timing_gen_index()
    #"""


...

[0.0978 0.2017 0.3956 0.305 ]
Timing fin2inf:
1000 loops, best of 3: 11.6 µs per loop
Timing inf2fin:
1000 loops, best of 3: 9.3 µs per loop
Timing genindex:



----------------------------------------------------------------------
Ran 3 tests in 0.468s

OK


1000 loops, best of 3: 53.9 µs per loop


In [3]:
#Class definition
class Angle(object):
    Deg=np.pi/180
    Rad=1/Deg
    
    def calcTrig(angle):
        """
        Parameters:
            angle: angle, float, radians
        Return:
            cos(angle), sin(angle): common trig. functions, tuple (2), adimensiona
        """
        return np.cos(angle),np.sin(angle)

    def dms(value):
        """
        Parameters:
            dec: Angle in decimal, float, degrees
        Return:
            dms: Angle in dms, tuple/list/array(4), (sign,deg,min,sec)
        """
        sgn=np.sign(value)
        val=np.abs(value)
        deg=np.floor(val)
        rem=(val-deg)*60
        min=np.floor(rem)
        sec=(rem-min)*60
        return (sgn,deg,min,sec)
    
    def dec(dms):
        """
        Parameters:
            dms: Angle in dms, tuple/list/array(4), (sign,deg,min,sec)
        Return:
            dec: Angle in decimal, float, degree
        """
        return dms[0]*(dms[1]+dms[2]/60.0+dms[3]/3600.0)

    
#Unitary test
class Test(unittest.TestCase):

    def test_calc_trig(self):
        self.assertEqual(np.isclose(Angle.calcTrig(30.0*Angle.Deg),
                                    [np.sqrt(3)/2,1./2],
                                    rtol=1e-17).tolist(),
                          [True,True])

    def test_dms(self):
        dms=Angle.dms(293.231241241)
        self.assertEqual(np.isclose(dms,
                                    (1.0,293.0,13.0,52.46846760007429),
                                    rtol=1e-5).tolist(),
                         [True]*4)

    def test_dms_negative(self):
        dms=Angle.dms(-293.231241241)
        self.assertEqual(np.isclose(dms,
                                    (-1.0,293.0,13.0,52.46846760007429),
                                    rtol=1e-5).tolist(),
                         [True]*4)
        
    def test_dec(self):
        dec=Angle.dec((1,5,40,3.4567))
        self.assertAlmostEqual(dec,5.66762686,5)

    def test_dec_negative(self):
        dec=Angle.dec((-1,5,40,3.4567))
        self.assertAlmostEqual(dec,-5.66762686,5)

    def timing_calc_trig(self):
        Angle.calcTrig(30.0*Angle.Deg)
    
if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    print("Trigonometric functions:")
    %timeit -n 1000 Test().timing_calc_trig()

.....

Trigonometric functions:
1000 loops, best of 3: 10.7 µs per loop



----------------------------------------------------------------------
Ran 5 tests in 0.603s

OK


In [4]:
#Class definition
class Const(object):
    #Astronomical
    au=1.4959787070000000e8 #km, value assumed in DE430
    
    #Time
    Min=60.0 # seconds
    Hour=60.0*Min
    Day=24.0*Hour
    Year=365.24*Day
    SideralMonth=27.321661*Day
    
    #Length
    km=1000.0 # m
    au=1.4959787070000000e8*km
    
    #Speed
    kms=1000.0 # m/s
    
    #Units transformation
    def transformState(state,factors,implicit=False):
        """
        Change units of a state vector 
        Parameters:
            state: state vector (x,y,z,vx,vy,vz), float (6), (L,L,L,L/T,L/T,L/T)
            [facLen,facVel]: convesion factors, float (2)
        Return:
            state: converted state vector x*facLen,y*facLen,z*facLen,z*facLen,vx*facVel,vy*facVel,vz*facVel
                    float(6),(L,L,L,L/T,L/T,L/T)
        """
        facLen,facVel=factors
        if implicit:
            state[:3]*=facLen
            state[3:]*=facVel
        else:
            return np.concatenate((state[:3]*facLen,state[3:]*facVel))

    #Orbital elements
    def transformElements(elements,factors,implicit=False):
        """
        Change units of an elements vector
        Parameters:
            elements: elements vector (a,e,i,W,w,M), float (6), (L,1,RAD,RAD,RAD,RAD)
            [facLen,facAng]: convesion factors (length, angles), float (2)
        Return:
            elements: converted elements vector a*facLen,e,i*facAng,W*facAng,w*facAng,M*facAng
                    float(6),(L,L,L,L/T,L/T,L/T)
        """
        facLen,facAng=factors
        if implicit:
            elements[:1]*=facLen
            elements[2:]*=facAng
        else:
            return np.concatenate((elements[:1]*facLen,[elements[1]],elements[2:]*facAng))

#Unitary test
class Test(unittest.TestCase):
    
    state=np.array([-2.75666323e+07,1.44279062e+08,3.02263967e+04,
                    -2.97849475e+01,-5.48211971e+00,1.84565202e-05])
    elements=np.array([Const.au,0.5,10.0,30.0,60.0,120.0])
    
    def test_year(self):
        self.assertEqual(Const.Year,31556736.0)
        
    def test_au(self):
        self.assertEqual(Const.au,1.4959787070000000e11)

    def test_transform_state(self):
        Const.transformState(self.state,[Const.km,Const.kms],implicit=True)
        self.assertEqual(np.isclose(self.state,
                                    [-2.75666323e+10,1.44279062e+11,3.02263967e+07,
                                     -2.97849475e+04,-5.48211971e+03,1.84565202e-02],
                                    rtol=1e-5).tolist(),
                          [True]*6)
        
    def test_transform_elements(self):
        Const.transformElements(self.elements,[1/Const.au,Angle.Deg],implicit=True)
        self.assertEqual(np.isclose(self.elements,
                                    [1.,0.5,0.17453293,0.52359878,1.04719755,2.0943951],
                                    rtol=1e-5).tolist(),
                          [True]*6)
    
    def timing_trans_state(self):
        Const.transformState(self.state,[Const.km,Const.kms])
        
    def timing_trans_elements(self):
        Const.transformElements(self.elements,[1/Const.au,Angle.Deg])
    
    
if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    print("Timing year constant:")
    %timeit -n 1000 Const.Year
    
    print("Timing elements transformation:")
    %timeit -n 1000 Test().timing_trans_elements()
    
    print("Timing state transformation:")        
    %timeit -n 1000 Test().timing_trans_state()

....
----------------------------------------------------------------------
Ran 4 tests in 0.012s

OK


Timing year constant:
1000 loops, best of 3: 79.8 ns per loop
Timing elements transformation:
1000 loops, best of 3: 20.5 µs per loop
Timing state transformation:
1000 loops, best of 3: 15.2 µs per loop


In [5]:
#Class definition
class Spice(object):

    #System constants
    _KERNELDIR=f"util/kernels"
    
    #Shapes 
    Ra=dict()
    Rb=dict()
    Rc=dict()
    f=dict()
    
    #Gravitational constants / masses
    #https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/a_old_versions/de421_announcement.pdf
    Mu=dict(
        SSB=132712440040.944000*Const.km**3, #km^3/s^2
        SUN=132712440040.944000*Const.km**3, #km^3/s^2
        EARTH=398600.436233*Const.km**3, #km^3/s^2
        MOON=4902.800076*Const.km**3, #km^3/s^2
        EARTH_BARYCENTER=403503.236310*Const.km**3, #km^3/s^2
    )

    #Rotational Periods
    Prot=dict(
        SSB=27*Const.Day,
        SUN=27*Const.Day,
        EARTH=1*Const.Day,
        MOON=1*Const.SideralMonth,
    )
    
    #Reference Frames
    RF=dict(
        SSB="ECLIPJ2000",
        SUN="ECLIPJ2000",
        EARTH="ITRF93",
        MOON="IAU_MOON",
    )
    
    #Center
    Master=dict(
        SSB=[None,0],
        SUN=[None,0],
        EARTH=["SSB",Const.au],
        MOON=["EARTH",384000*Const.km]
    )
    
    def loadKernels():
        """
            Load Kernels
        """
        #Kernel sources: https://naif.jpl.nasa.gov/pub/naif/generic_kernels/
        kernels=[
            #Udates: https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/
            "naif0012.tls",
            #Updates: https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/
            "pck00010.tpc",
            #For updates and docs: https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/
            "de430.bsp",
            #Updates: https://naif.jpl.nasa.gov/pub/naif/generic_kernels/pck/
            "earth_720101_070426.bpc",
            "earth_070425_370426_predict.bpc",
            "earth_latest_high_prec_20190910.bpc",
            "moon_pa_de421_1900-2050.bpc",
            "moon_080317.tf"
            ]
        for kernel in kernels:
            spy.furnsh(f"{Spice._KERNELDIR}/{kernel}")
            
    def calcShape(objid):
        """
        Calculate shape of objetc objid.
        Parameters:
            objid: name of the object (eg. EARTH, MOON, etc.), string
        Return: None
        """
        try:
            Ra,Rb,Rc=spy.bodvrd(objid,"RADII",3)[1]
        except:
            Ra=Rb=Rc=1
            
        Spice.Ra[objid]=Ra*Const.km
        Spice.Rb[objid]=Rb*Const.km
        Spice.Rc[objid]=Rc*Const.km
        Spice.f[objid]=(Ra-Rc)/Ra
            
    def str2t(date):
        """
        Convert date from string TDB to TDB 
        Parameters:
            date: date string in TDB (eg. CCYY Mmm DD HH:HH:HH), string
        Returns:
            tdb: tdb, float, seconds since 2000 JAN 01 12:00:00 TDB.
        """
        et=spy.str2et(date)
        dt=spy.deltet(et,"ET")
        t=et-dt
        return t

#Unitary test
class Test(unittest.TestCase):

    def test_load_kernels(self):
        Spice.loadKernels()
        
    def test_shape(self):
        Spice.calcShape("EARTH")
        self.assertEqual(np.isclose([Spice.Ra["EARTH"],Spice.f["EARTH"]],
                                    [6378.1366e3,0.0033528131084554717],
                                    rtol=1e-5).tolist(),
                          [True,True])
        
    def test_string_tdb(self):
        self.assertAlmostEqual(Spice.str2t("2000 JAN 01 12:00:00"),0.0,7)
        
    def test_right_kernels(self):
        pass
    
    def timing_load_kernels(self):
        Spice.loadKernels()
        
if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    print("Timing loadKernels:")
    %timeit -n 10 Test().timing_load_kernels()

....
----------------------------------------------------------------------
Ran 4 tests in 0.119s

OK


Timing loadKernels:
10 loops, best of 3: 71.8 ms per loop


In [6]:
#Class definition
class Body(object):
    """
    Define a body
    
    Atributes:
        objid: String with name of object (eg. MOON), string 
        refid: String with name of reference frame (eg. IAU_MOON), string
        P (optional): Rotational period, float, seconds
    """
    state=np.zeros(6)
    Tbod2ecl=np.zeros((3,3))
    
    def __init__(self,objid):
        self.id=objid
        
        if self.id is None:
            raise AssertionError("Body id is None")
        
        #Get geometrical, gravitational and physical properties
        self.rf=Spice.RF[self.id]
        self.master,self.amaster=Spice.Master[self.id]
        Spice.calcShape(self.id)
        self.Ra=Spice.Ra[self.id]
        self.Rb=Spice.Rb[self.id]
        self.Rc=Spice.Rc[self.id]
        self.f=Spice.f[self.id]
        self.mu=Spice.Mu[self.id]
        self.Prot=Spice.Prot[self.id]
        
        #Derivative
        if self.master is not None:
            self.rhill=self.amaster*(self.mu/(3*Spice.Mu[self.master]))**(1./3)
        else:
            self.rhill=1
            
    def updateBody(self,tdb):
        self.tdb=tdb
        self.state,self.tlight=spy.spkezr(self.id,tdb,"ECLIPJ2000","NONE","SSB")
        self.stateHelio=Const.transformState(self.state,[Const.km,Const.kms])
        self.Tbod2ecl=spy.pxform(self.rf,"ECLIPJ2000",tdb)
        self.Tecl2bod=np.linalg.inv(self.Tbod2ecl)

#Unitary test
class Test(unittest.TestCase):
    
    tdb=Spice.str2t("2000 JAN 01 12:00:00")
    earth=Body("EARTH")
    moon=Body("MOON")

    def test_body_rhill(self):
        self.assertEqual(np.isclose([self.earth.rhill],[1496558526],rtol=1e-5).tolist(),[True])

    def test_moon_rhill(self):
        self.assertEqual(np.isclose([self.moon.rhill],[61460054],rtol=1e-5).tolist(),[True])

    def test_body_shape(self):
        Spice.calcShape("EARTH")
        self.assertEqual(np.isclose([self.earth.Ra,self.earth.f],
                                    [6378.1366e3,0.0033528131084554717],
                                    rtol=1e-5).tolist(),
                          [True,True])
            
    def test_body_state(self):
        self.earth.updateBody(self.tdb)
        self.assertEqual(np.isclose(self.earth.stateHelio,
                                    [-2.75666323e+10,1.44279062e+11,3.02263967e+07,
                                     -2.97849475e+04,-5.48211971e+03,1.84565202e-02],
                                    rtol=1e-5).tolist(),
                          [True]*6)

    def test_body_transform(self):
        self.earth.updateBody(self.tdb)
        self.assertEqual(np.isclose(self.earth.Tbod2ecl.flatten(),
                                    [ 1.76980593e-01,9.84214341e-01,-2.51869708e-05,
                                     -9.03007988e-01,1.62388314e-01,3.97751944e-01,
                                     3.91477257e-01,-7.03716309e-02,9.17492992e-01],
                                    rtol=1e-5).tolist(),
                          [True]*9)
    
    def test_moon_state(self):
        self.moon.updateBody(self.tdb)
        self.assertEqual(np.isclose(self.moon.stateHelio,
                                    [-2.78582406e+10,1.44004083e+11,6.64975943e+07,
                                     -2.91414161e+04,-6.21310369e+03,-1.14880075e+01],
                                    rtol=1e-5).tolist(),
                          [True]*6)
        self.assertEqual(np.isclose(self.moon.Tbod2ecl.flatten(),
                                    [0.78422705,-0.62006192,-0.02260867,
                                     0.61987147,0.78455064,-0.01548052,
                                     0.02733653,-0.00187423,0.99962453],
                                    rtol=1e-5).tolist(),
                          [True]*9)
    
    def timing_update(self):
        self.earth.updateBody(self.tdb)
    
if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    print("Timing update body:")
    %timeit -n 100 Test().timing_update()


......

Timing update body:
100 loops, best of 3: 127 µs per loop



----------------------------------------------------------------------
Ran 6 tests in 0.073s

OK


In [7]:
#Class definition
class Location(object):
    """
    Define a location on a scenario
    
    Atributtes:
        scenario: Scenario of the location, Scenario
        longitud: longitude, float, radians
        latitude: latitude, float, radians
        altitude: elevation over reference ellipsoid, float, m
    
    """
    
    def __init__(self,body,longitude,latitude,altitude):
        self.body=body
        self.lon=longitude
        self.lat=latitude
        self.alt=altitude
        
        self.posBody=spy.georec(self.lon,self.lat,self.alt,
                                self.body.Ra,self.body.f) 

        #Position of the location w.r.t. to itself (added for consistency)
        self.posLocal=np.zeros(3) 
        
        #Velocity local is the surface velocity due to planetary rotation: 2 pi rho/P
        rho=((self.posBody[:2]**2).sum())**0.5
        self.velLocal=np.array([0,+2*np.pi*rho/self.body.Prot,0])
        
        #Transformation matrix from local to body and viceversa
        uz=spy.surfnm(self.body.Ra,self.body.Rb,self.body.Rc,self.posBody)
        uy=spy.ucrss(np.array([0,0,1]),uz)
        ux=spy.ucrss(uz,uy)
        self.Tloc2bod=np.array(np.vstack((ux,uy,uz)).transpose().tolist())
        self.Tbod2loc=np.linalg.inv(self.Tloc2bod)
        
        #Velocity of the surface with respect to the inertial ref. frame of the body
        self.velBody=spy.mxv(self.Tloc2bod,self.velLocal)
    
    def updateLocation(self,tdb_moon):
        
        self.body.updateBody(tdb_moon)
        
        #Position of the location in the Ecliptic reference system
        self.posEcl=spy.mxv(self.body.Tbod2ecl,self.posBody)
        
        #Velocity of the location in the Ecliptic reference system
        self.velEcl=spy.mxv(self.body.Tbod2ecl,self.velBody)
    
    def vbod2loc(self,vBod):
        """
        Parameters:
            vBod: Vector in the body system, numpy array (3)
        Return:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
        """
        vLoc=spy.mxv(self.Tbod2loc,vBod)
        vimp,A,h=spy.reclat(vLoc)
        A=2*np.pi+A if A<0 else A
        return A,h,vimp

    def loc2vbod(self,A,h,v):
        """
        Express a vector in the direction A,h with magnitude v in the rotating
        reference frame of the central object of the location.
        
        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
            v: Vector magnitude, (-infty,infty), float, (arbitrary)
               If v<0 then the vector points in the opposite direction of (A,h)
        Return:
            vBod: Velocity in the body-fixed system, np.array, km/s
        """
        vLoc=spy.latrec(v,A,h)
        vBod=spy.mxv(self.Tloc2bod,vLoc)
        return vBod
    
    def ecl2loc(self,eclon,eclat):
        """
        Parameters:
            eclon: Ecliptic longitude, float, radians
            eclat: Ecliptic latitude, float, radians
        Return:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
        
        NOTE: It requires to run previously the update method.
        """
        ecx,ecy,ecz=spy.latrec(1,eclon,eclat)
        x,y,z=spy.mxv(self.Tbod2loc,spy.mxv(self.body.Tecl2bod,[ecx,ecy,ecz]))
        r,A,h=spy.reclat([x,y,z])
        A=2*np.pi+A if A<0 else A
        return A,h

    def loc2ecl(self,A,h):
        """
        Parameters:
            A: Azimuth (0,2 pi), float, radians
            h: Elevation (-pi,pi), float, radians
        Return:
            eclon: Ecliptic longitude (0,2pi), float, radians
            eclat: Ecliptic latitude (-pi,pi), float, radians
            
        NOTE: It requires to run previously the update method.
        """
        x,y,z=spy.latrec(1,A,h)
        ecx,ecy,ecz=spy.mxv(self.body.Tbod2ecl,spy.mxv(self.Tloc2bod,[x,y,z]))
        r,eclon,eclat=spy.reclat([ecx,ecy,ecz])
        eclon=2*np.pi+eclon if eclon<0 else eclon
        return eclon,eclat
        
#Unitary test
class Test(unittest.TestCase):

    #Objects
    moon=Body("MOON")
    earth=Body("EARTH")

    #Moon impact
    tdb_moon=spy.str2et("2000 JAN 02 11:58:56 UTC")
    crater=Location(moon,45.6452*Angle.Deg,41.1274*Angle.Deg,10.0*Const.km)

    #Earth impact
    tdb_earth=spy.str2et("2000 JAN 01 12:00:00 UTC")
    impact=Location(earth,0*Angle.Deg,0*Angle.Deg,0*Const.km)
    
    #Chelyabinsk impact
    tdb_chely=spy.str2et("02/15/2013 3:20:34 UTC")
    chely=Location(earth,61.1*Angle.Deg,54.8*Angle.Deg,23.3*Const.km)
    
    def test_vbod2loc(self):
        #These are the components of the Chelyabinsk-impactor velocity as reported by CNEOS
        vBod=np.array([12.8,-13.3,-2.4])
        A,h,vimp=self.chely.vbod2loc(-vBod)
        self.assertAlmostEqual(A*Angle.Rad,99.8961127649985,5)
        self.assertAlmostEqual(h*Angle.Rad,15.92414245029081,5)
        
    def test_loc2vbod(self):
        #These is the radiant and speed (18.6 km/s) of the chelyabinsk impact 
        vBod=self.chely.loc2vbod(101.1*Angle.Deg,+15.9*Angle.Deg,-18.6)
        self.assertEqual(np.isclose(vBod,
                                    np.array([12.8,-13.1,-2.4]),
                                    rtol=1e-1).tolist(),
                         [True]*3)
        
    def test_earth_impact(self):
        self.impact.updateLocation(self.tdb_earth)

        #Body and location properties
        self.assertEqual(np.isclose(self.impact.Tloc2bod.flatten(),
                                    [0.,0.,1.,0.,1.,0.,1.,0.,0.],
                                    rtol=1e-5).tolist(),
                          [True]*9)        
        self.assertEqual(np.isclose(self.impact.posLocal,
                                    [0.,0.,0.],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.impact.velLocal,
                                    [0.,463.83118255,0.],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.impact.posBody,
                                    [6378136.6,0.,0.],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.impact.velBody,
                                    [0.,463.83118255,0.],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.impact.posEcl,
                                    [1158174.70642354,-5754597.59496353,2494767.39551009],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.impact.velEcl,
                                    [456.12009587,77.28027145,-33.49005365],
                                    rtol=1e-5).tolist(),
                          [True]*3)

        #Position of the Sun at the date of the test
        eclon=Angle.dec((+1,280,22,21.9))
        eclat=Angle.dec((-1,0,0,2.7))
        A,h=self.impact.ecl2loc(eclon*Angle.Deg,eclat*Angle.Deg)

        #Position of Betelgeuse
        eclon=Angle.dec((+1,88,45,16.6))
        eclat=Angle.dec((-1,16,1,37.2))
        A,h=self.impact.ecl2loc(eclon*Angle.Deg,eclat*Angle.Deg)
        self.assertAlmostEqual(A*Angle.Rad,57.27518638612843,5)
        self.assertAlmostEqual(h*Angle.Rad,-76.20677246845091,5)

        A=Angle.dec((+1,57,16,30.7))
        h=Angle.dec((-1,76,12,24.4))
        eclon,eclat=self.impact.loc2ecl(A*Angle.Deg,h*Angle.Deg)
        self.assertAlmostEqual(eclon*Angle.Rad,88.75461469860417,5)
        self.assertAlmostEqual(eclat*Angle.Rad,-16.027004471139914,5)        
       
    def test_moon_loc2bod(self):
        self.assertEqual(np.isclose(self.crater.Tloc2bod.flatten(),
                                    [-0.45982258,-0.71502441,0.52659594,
                                     -0.47029697,0.69909948 ,0.53859138,
                                      0.75324894,0.,0.65773554],
                                    rtol=1e-5).tolist(),
                          [True]*9)
    
    def test_moon_local(self):
        self.assertEqual(np.isclose(self.crater.posLocal,
                                    [0.,0.,0.],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.crater.velLocal,
                                    [0.,3.50340129,0.],
                                    rtol=1e-5).tolist(),
                          [True]*3)
    
    def test_moon_body(self):
        self.assertEqual(np.isclose(self.crater.posBody,
                                    [920173.74904705,941134.57557525,1149327.08234927],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.crater.velBody,
                                    [-2.50501745,2.44922603,0.],
                                    rtol=1e-5).tolist(),
                          [True]*3)

    def test_moon_ecliptic(self):
        self.crater.updateLocation(self.tdb_moon)
        self.assertEqual(np.isclose(self.crater.posEcl,
                                    [-189857.68536427,1287964.72727012,1165550.37669598],
                                    rtol=1e-5).tolist(),
                          [True]*3)
        self.assertEqual(np.isclose(self.crater.velEcl,
                                    [-3.47523194,-0.43509669,-0.08529044],
                                    rtol=1e-5).tolist(),
                          [True]*3)

    def test_moon_ecl2loc(self):
        self.crater.updateLocation(self.tdb_moon)
        eclon=25.3157371
        eclat=-1.2593327
        A,h=self.crater.ecl2loc(eclon*Angle.Deg,eclat*Angle.Deg)
        self.assertEqual(np.isclose([A*Angle.Rad,h*Angle.Rad],
                                    [255.7181,11.6614],
                                    atol=1e-2).tolist(),
                         [True]*2)

    def test_moon_loc2ecl(self):
        self.crater.updateLocation(self.tdb_moon)
        A=229.2705
        h=28.8062
        eclon,eclat=self.crater.loc2ecl(A*Angle.Deg,h*Angle.Deg)
        self.assertEqual(np.isclose([eclon*Angle.Rad,eclat*Angle.Rad],
                                    [55.1580499,-5.0588748],
                                    atol=1e-2).tolist(),
                         [True]*2)
    
    def timing_update(self):
        self.crater.updateLocation(self.tdb_moon)

if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    print("Timing update location:")
    %timeit -n 100 Test().timing_update()

.........

Timing update location:
100 loops, best of 3: 202 µs per loop



----------------------------------------------------------------------
Ran 9 tests in 0.021s

OK


In [309]:
#Class definition
class KeplerianOrbit(object):
    """
    A keplerian orbit
    
    Input attributes:
        mu: Gravitational parameter

    Other attributes:
        elements: cometary elements
            a: semimajor axis (-inf<a<inf), float, length
            e: eccentricity (e>=0), float
            i: inclination (0<i<pi), float, radians
            W: longitude of the ascending node (0<W<2pi), float, radians
            w: argument of the periapsis (0<w<2pi), float, radians
            M: mean anomaly (0<M<2pi), float, radians
    
        celements: classical elements:
            a: semimajor axis (-inf<a<inf), float, length
            e: eccentricity (e>=0), float
            i: inclination (0<i<pi), float, radians
            W: longitude of the ascending node (0<W<2pi), float, radians
            w: argument of the periapsis (0<w<2pi), float, radians
            M: mean anomaly (0<M<2pi), float, radians

        s: signatue (+1 ellipse,-1 hyperbola, 0 parabolla)
            
        state: state vector (x,y,z,vx,vy,vz), numpy array (6), L,L,L,L/T,L/T,L/T
        
        uelements: unbound elements: A, E, I, O, W, M

        Derivative properties: Other elements:
            n: mean motion (n>0), float, 1/T
            ab: |a| (|a|>0), float, length
            eps: Eccentricity parameter, eps=sqrt[s(1-e**2)] (eps>=0), float
            b: semiminor axis, ab*eps
            cosE: "cosine" of the eccenric anomaly, cos(E) or cosh(H)
            sinE: "sine" of the eccenric anomaly, sin(E) or sinh(H)

    """
    s=0
    scales=[1,1,np.pi,np.pi,np.pi,np.pi]
    elements=np.zeros(6)
    celements=np.zeros(6)
    uelements=np.zeros(6)
    state=np.zeros(6)
    derivatives=[]
    
    def __init__(self,mu):
        self.mu=mu
        
    def setElements(self,elements,t):
        self.elements=elements
        self.t=t
        if self.elements[1]>1:
            self.s=-1
        else:
            self.s=+1
        self.state=spy.conics(list(self.elements)+[t,self.mu],t)
        np.copyto(self.celements,self.elements)
        self.celements[0]=self.elements[0]/(1-self.elements[1])
        self.calcDerivatives()
    
    def setUelements(self,uelements,t,maxvalues=[1.0,1.0,np.pi,2*np.pi,2*np.pi,2*np.pi]):
        elements=np.array([Util.inf2Fin(uelements[i],maxvalues[i]) for i in range(6)])
        self.setElements(elements,t)
        
    def setState(self,state,t):
        self.state=state
        self.t=t
        elements=spy.oscelt(self.state,t,self.mu)
        self.elements=elements[:6]
        np.copyto(self.celements,self.elements)
        self.celements[0]=self.elements[0]/(1-self.elements[1])
        if self.elements[1]>1:
            self.s=-1
        else:
            self.s=+1
        self.calcDerivatives()        
        
    def updateState(self,t):
        self.state=spy.conics(list(self.elements)+[self.t,self.mu],t)
        #Update derivatives
        self.calcDerivatives()
        #Update M (it can be improved): M = s (E - e sinE) (where sinE is sinhE in case of e>1)
        self.celements[-1]=self.elements[-1]=spy.oscelt(self.state,t,self.mu)[5]
        self.t=t
        
    def calcDerivatives(self):

        #Get elements and state vector
        a,e,i,W,w,M=self.celements
        q=self.elements[0]
        x,y,z,vx,vy,vz=self.state
        mu=self.mu
        s=self.s
        r=(x**2+y**2+z**2)**0.5

        #Auxiliar
        cosi,sini=Angle.calcTrig(i)
        cosw,sinw=Angle.calcTrig(w)
        cosW,sinW=Angle.calcTrig(W)
        C=(cosw*sinW+sinw*cosi*cosW);D=(-sinw*sinW+cosw*cosi*cosW)
        
        #Derivatives
        ab=np.abs(a)
        n=np.sqrt(mu/ab**3)
        eps=np.sqrt(s*(1-e**2))
        b=ab*eps
        cosE=(1/e)*(1-r/a)
        sinE=(y-a*(cosE-e)*C)/(ab*eps*D)
        
        self.derivatives=np.array([n,ab,eps,b,cosE,sinE])
    
    def calcUelements(self,maxvalues=[1.0,1.0,np.pi,2*np.pi,2*np.pi,2*np.pi]):
        self.uelements=np.array([Util.fin2Inf(self.elements[i],maxvalues[i]) for i in range(6)])
    
    
    def calcJacobians(self):
        """
        Compute the Jacobian Matrix of the transformation from classical 
        orbital elements (a,e,i,w,W,M) to cartesian state vector (x,y,z,x',y',z').

        Return:

            Jc2k = [dx/da,dx/de,dx/di,dx/dw,dx/dW,dx/dM,
                    dy/da,dy/de,dy/di,dy/dw,dy/dW,dy/dM,
                    dz/da,dz/de,dz/di,dz/dw,dz/dW,dz/dM,
                    dx'/da,dx'/de,dx'/di,dx'/dw,dx'/dW,dx'/dM,
                    dy'/da,dy'/de',dy'/di,dy'/dw,dy'/dW,dy'/dM,
                    dz'/da,dz'/de,dz'/di',dz'/dw,dz'/dW,dz'/dM],

                    Numpy array 6x6, units compatible with mu and a.
        """
        a,e,i,W,w,M=self.celements
        q=self.elements[0]
        mu=self.mu
        s=self.s
        
        #Trigonometric function
        cosi,sini=Angle.calcTrig(i)
        cosw,sinw=Angle.calcTrig(w)
        cosW,sinW=Angle.calcTrig(W)

        #Components of the rotation matrix
        A=(cosW*cosw-cosi*sinW*sinw);B=(-cosW*sinw-cosw*cosi*sinW)
        C=(cosw*sinW+sinw*cosi*cosW);D=(-sinw*sinW+cosw*cosi*cosW)
        F=sinw*sini;G=cosw*sini

        #Primary auxiliar variables
        ab=np.abs(a)
        n=np.sqrt(mu/ab**3)
        nu=n*a**2
        eps=np.sqrt(s*(1-e**2))

        #Get cartesian coordinates
        x,y,z,vx,vy,vz=self.state
        r=(x**2+y**2+z**2)**0.5
        nur=nu/r

        #Eccentric anomaly as obtained from indirect information
        #From the radial equation: r = a (1-e cos E)
        cosE=(1/e)*(1-r/a)

        #From the general equation for y
        #NOTE: This is the safest way to obtain sinE without the danger of singularities
        sinE=(y-a*(cosE-e)*C)/(ab*eps*D)

        #dX/da
        Ja=np.array([x/a,y/a,z/a,-vx/(2*a),-vy/(2*a),-vz/(2*a)])

        #dX/de
        dcosEde=-s*a*sinE**2/r
        dsinEde=a*cosE*sinE/r
        dnurde=(nu*a/r**2)*(cosE-(ab/r)*e*sinE**2)
        depsde=-s*e/eps

        drAde=a*(dcosEde-1)
        drBde=ab*(depsde*sinE+eps*dsinEde)

        dvAde=-(dnurde*sinE+nur*dsinEde)
        dvBde=(dnurde*eps*cosE+nur*depsde*cosE+nur*eps*dcosEde)

        Je=np.array([
            drAde*A+drBde*B,
            drAde*C+drBde*D,
            drAde*F+drBde*G,
            dvAde*A+dvBde*B,
            dvAde*C+dvBde*D,
            dvAde*F+dvBde*G,
        ])

        #dX/di
        Ji=np.array([z*sinW,-z*cosW,-x*sinW+y*cosW,vz*sinW,-vz*cosW,-vx*sinW+vy*cosW])

        #dX/dw
        Jw=np.array([-y*cosi-z*sini*cosW,x*cosi-z*sini*sinW,sini*(x*cosW+y*sinW),\
            -vy*cosi-vz*sini*cosW,vx*cosi-vz*sini*sinW,sini*(vx*cosW+vy*sinW)])

        #dX/dW
        JW=np.array([-y,x,0,-vy,vx,0])

        #dX/dM
        JM=ab**1.5*np.array([vx,vy,vz,-x/r**3,-y/r**3,-z/r**3])

        #Jacobian
        self.Jck=np.array([Ja,Je,Ji,Jw,JW,JM]).transpose()
        self.Jkc=np.linalg.inv(self.Jck)
    
    def calcJacobiansMap(self):
        """
        Parameters:
            epsilon: bound elements, numpy array (N)
            scales: scales for the bound elements ()


        Return:

            Jif= [dE_1/de_1,        0,        0,...,        0,
                          0,dE_2/de_2,        0,...,        0,
                          0,        0,dE_2/de_2,...,        0,
                                 . . . 
                          0,        0,        0,...,dE_N/de_N]

            where dE/de = (1/s) /[x(1-x)] and x = e/s.
        """
        self.JEe=np.identity(6)
        self.JeE=np.identity(6)
        for i,eps in enumerate(self.elements):
            x=eps/self.scales[i]
            self.JEe[i,i]=(1/self.scales[i])/(x*(1-x))
            self.JeE[i,i]=1/self.JEe[i,i]
    
#Unitary test
class Test(unittest.TestCase):
    
    #Orbit
    orbit=KeplerianOrbit(1.0)
    
    #"""
    def test_jacobians_Ee(self):
        self.timing_set_bound_ellipse()
        self.orbit.calcJacobiansMap()
        self.assertEqual(
            np.isclose(np.diag(self.orbit.JEe),
                       [6.25,4.16666667,1.69765273,1.43239449,2.29183118,3.2228876],
                       rtol=1e-5).tolist(),
            [True]*6
        )

    def test_set_uelements(self):
        self.orbit.setUelements([1.26566637,0.40546511,-1.09861229,-1.60943791,-2.39789527,-2.83321334],0.0)
        self.assertEqual(
            np.isclose(self.orbit.elements,
                       [0.78,0.6,0.78539816,1.04719755,0.52359878,0.34906585],
                       rtol=1e-5).tolist(),
            [True]*6
        )
    
    def test_uelements(self):
        mu=1
        q=0.78
        e=0.6
        i=45.0
        W=60.0
        w=30.0
        M=20.0
        self.orbit.setElements([q,e,i*Angle.Deg,W*Angle.Deg,w*Angle.Deg,M*Angle.Deg],0.0)
        self.orbit.calcUelements()
        self.assertEqual(
            np.isclose(self.orbit.uelements,
                       [1.26566637,0.40546511,-1.09861229,-1.60943791,-2.39789527,-2.83321334],
                       rtol=1e-5).tolist(),
            [True]*6
        )
        
    def test_update_state(self):
        self.timing_set_by_elements_ellipse()
        self.orbit.updateState(10.0)
        self.assertEqual(
            np.isclose(self.orbit.state,
                       [-1.86891304,-4.32426264,-0.54360515,0.1505539,-0.19731752,-0.22904226],
                       rtol=1e-5).tolist(),
            [True]*6
        )
        self.assertAlmostEqual(self.orbit.elements[-1],2.0558356849380304,7)
        self.assertAlmostEqual(self.orbit.celements[-1],2.0558356849380304,7)

    def test_by_elements_ellipse(self):
        self.timing_set_by_elements_ellipse()
        self.orbit.updateState(10.0)
        self.assertEqual(np.isclose(self.orbit.state,
                                    [-1.86891304,-4.32426264,-0.54360515,
                                     0.1505539,-0.19731752,-0.22904226],
                                    rtol=1e-5).tolist(),
                          [True]*6)
        self.assertEqual(np.isclose(self.orbit.derivatives[-2:],
                                    [-0.76518368,0.64381204],
                                    rtol=1e-5).tolist(),
                          [True]*2)
        
    def test_by_elements_hyperbola(self):
        self.timing_set_by_elements_hyperbola()
        self.assertAlmostEqual(self.orbit.celements[0],-2.16666667,7)
        self.assertEqual(np.isclose(self.orbit.state,
                                    [-1.01844517,0.74222555,1.25311217,
                                     -0.97533765,-0.56564603,0.56184417],
                                    rtol=1e-5).tolist(),
                          [True]*6)

    def test_by_elements_ellipse(self):
        self.timing_set_by_elements_ellipse()
        self.assertAlmostEqual(self.orbit.celements[0],3.25,7)
        self.assertEqual(np.isclose(self.orbit.state,
                                    [-1.35375627,0.1389311,1.24185287,
                                     -0.52685483,-0.69924506,0.10664714],
                                    rtol=1e-5).tolist(),
                          [True]*6)

    def test_Jacobian(self):
        self.timing_set_by_elements_ellipse()
        self.orbit.calcJacobians()
        self.assertEqual(np.isclose(self.orbit.Jck.flatten(),
                                    [-0.41654039,-1.23208195,1.07547613,-0.53730042,-0.1389311,-3.08685343,
                                      0.04274803,-5.63319668,-0.62092643,-1.7177267,-1.35375627,-4.09689136,
                                      0.38210857,-1.74958408,1.24185287,-0.39354754,0.,0.62484781,
                                      0.08105459,0.6202556,0.09235913,0.45673547,0.69924506,1.26843361,
                                      0.10757616,-0.91335224,-0.05332357,-0.43785039,-0.52685483,-0.13017475,
                                     -0.01640725,-0.99383322,0.10664714,-0.61446971,0.,-1.1635831],
                                    rtol=1e-5).tolist(),
                          [True]*36)

    def test_derivatives(self):
        self.timing_set_by_elements_ellipse()
        self.orbit.calcDerivatives()
        self.assertEqual(np.isclose(self.orbit.derivatives,
                                    [0.17067698,3.25,0.8,2.6,0.72188531,0.69201272],
                                    rtol=1e-5).tolist(),
                          [True]*6)
        
    #"""
    
    def timing_set_by_state_hyperbola(self):
        state=[-1.02,0.74,1.25,-0.98,-0.56,0.56]
        self.orbit.setState(state,0.0)
     
    def timing_set_by_elements_hyperbola(self):
        mu=1
        q=1.3
        e=1.6
        i=45.0
        w=30.0
        W=60.0
        M=20.0
        self.orbit.setElements([q,e,i*Angle.Deg,W*Angle.Deg,w*Angle.Deg,M*Angle.Deg],0.0)

    def timing_set_by_elements_ellipse(self):
        mu=1
        q=1.3
        e=0.6
        i=45.0
        w=30.0
        W=60.0
        M=20.0
        self.orbit.setElements([q,e,i*Angle.Deg,W*Angle.Deg,w*Angle.Deg,M*Angle.Deg],0.0)

    def timing_set_bound_ellipse(self):
        mu=1
        q=0.8
        e=0.6
        i=45.0
        w=30.0
        W=60.0
        M=20.0
        self.orbit.setElements([q,e,i*Angle.Deg,W*Angle.Deg,w*Angle.Deg,M*Angle.Deg],0.0)

if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

    #"""
    print("Timing set by elements hyperbola:")
    %timeit -n 100 Test().timing_set_by_elements_hyperbola()

    print("Timing set by elements ellipse:")
    %timeit -n 100 Test().timing_set_by_elements_ellipse()

    print("Timing set by state hyperbola:")
    %timeit -n 100 Test().timing_set_by_state_hyperbola()

    print("Timing Jacobian Jck:")
    t=Test()
    %timeit -n 100 t.orbit.calcJacobians()
    
    print("Timing Determinant of Jacobian:")
    %timeit -n 100 np.linalg.det(t.orbit.Jck)

    print("Timing update state:")
    t=Test()
    %timeit -n 1000 t.orbit.updateState(10.0)

    print("Timing Jacobians Ee:")
    t=Test()
    t.timing_set_bound_ellipse()
    %timeit -n 100 t.orbit.calcJacobiansMap()
    #"""


........
----------------------------------------------------------------------
Ran 8 tests in 0.045s

OK


Timing set by elements hyperbola:
100 loops, best of 3: 93.6 µs per loop
Timing set by elements ellipse:
100 loops, best of 3: 84 µs per loop
Timing set by state hyperbola:
100 loops, best of 3: 71.2 µs per loop
Timing Jacobian Jck:
100 loops, best of 3: 94.1 µs per loop
Timing Determinant of Jacobian:
100 loops, best of 3: 12.6 µs per loop
Timing update state:
1000 loops, best of 3: 109 µs per loop
Timing Jacobians Ee:
100 loops, best of 3: 22.8 µs per loop


In [310]:
#Class definition
class GrtRay(object):
    """
    A ray in a GRT analysis
    
    Input attributes:
        location: Location of the ray, object of class Location
        azimuth: Azimuth (0,2pi), float, radians
        elevation: Elevation (-pi/2,pi/2), float, radians
        speed: speed of the ray at the location (in the rotating reference frame), float, km/s
        
        NOTE: It is VERY importante to take into account that if a particle is COMING from A,h, 
              you need to specify its velocity as (A,h,-speed) or (360-A,-h,speed).
              
    Other attributes:
        scenario: Scenario where the ray propagates.
        body: Central body where the ray starts.
        
    """
    
    def __init__(self,location,azimuth,elevation,speed):
        
        #Attributes
        self.location=location
        self.body=self.location.body
        self.A=azimuth
        self.h=elevation
        self.vimp=speed
        
        #Instantiate masters
        master=self.body.master
        self.masters=dict()
        while master is not None:
            self.masters[master]=Body(master)
            master=self.masters[master].master

        #Body-centric state vector in the body axis
        self.velLoc=spy.latrec(self.vimp,self.A,self.h)
        self.velBody=spy.mxv(self.location.Tloc2bod,self.velLoc)+self.location.velBody
        self.stateBody=np.concatenate((self.location.posBody,self.velBody))

    def updateRay(self,tdb):
        self.tdb=tdb
        self.location.updateLocation(tdb)
        
        #Body-centric state vector in the ecliptic axis
        self.velEcl=spy.mxv(self.body.Tbod2ecl,self.velBody)
        self.stateEcl=np.concatenate((self.location.posEcl,self.velEcl))
        
        #Jacobian of the transformation (lon,lat,alt,A,h,vimp)->(x,y,z,x',y',z')
        #self.Jxgeo2rimp
    
    def calcJacobiansBody(self):
        """
        Compute the Jacobian Matrix of the transformation from 
        local impact conditions (lon,lat,alt,A,h,v) to cartesian state vector (x,y,z,x',y',z') 
        (in the body reference frame).


        Parameters:
            lon: Geographic longitude (0,2pi), float, radians
            lat: Geographic latitude (0,2pi), float, radians
            alt: Altitude over the ellipsoid (0,inf), float, km
            A: Azimuth (0,2pi), float, radians
            h: Elevation (-pi/2,pi/2), float, radians
            v: Impact speed (-inf,inf), float, km/s (negative if it is impacting)

        Return:

            Jc2l = [dx/dlon,dx/dlat,dx/dalt,dx/dA,dx/dh,dx/dv,
                    dy/dlon,dy/dlat,dy/dalt,dy/dA,dy/dh,dy/dv,
                    dz/dlon,dz/dlat,dz/dalt,dz/dA,dz/dh,dz/dv,
                    dx'/dlon,dx'/dlat,dx'/dalt,dx'/dA,dx'/dh,dx'/dv,
                    dy'/dlon,dy'/dlat,dy'/dalt,dy'/dA,dy'/dh,dy'/dv,
                    dz'/dlon,dz'/dlat,dz'/dalt,dz'/dA,dz'/dh,dz'/dv],

                    Numpy 6x6 array.
        """

        #Local to rotating
        lon,lat,alt,A,h,vimp=self.location.lon,self.location.lat,self.location.alt,self.A,self.h,self.vimp
        x,y,z,vx,vy,vz=self.stateBody
        
        coslon,sinlon=Angle.calcTrig(lon)
        coslat,sinlat=Angle.calcTrig(lat)
        cosA,sinA=Angle.calcTrig(A)
        cosh,sinh=Angle.calcTrig(h)
        
        P=self.location.body.Prot
        a=self.location.body.Ra
        b=self.location.body.Rc
        
        #Auxiliar
        fr=2*np.pi*np.sqrt(x**2+y**2)/(P*vimp)
        N=a**2/np.sqrt(a**2*coslat**2+b**2*sinlat**2)
        n2=(2*np.pi/P)**2

        #dX/dlon:
        Jlon=np.array([-y,x,0,-vy,vx,0])

        #dX/dlat:
        dxdlat=(a**2-b**2)*coslat*sinlat*N**3/a**4*coslat*coslon-(N+alt)*sinlat*coslon
        dydlat=(a**2-b**2)*coslat*sinlat*N**3/a**4*coslat*sinlon-(N+alt)*sinlat*sinlon
        Jlat=np.array([
            dxdlat,
            dydlat,
            b**2*(a**2-b**2)*coslat*sinlat*N**3/a**6*sinlat+(b**2*N/a**2+alt)*coslat,
            -vimp*cosh*cosA*coslat*coslon-n2*sinlon/(fr*vimp)*(x*dxdlat+y*dydlat)-vimp*sinh*sinlat*coslon,
            -vimp*cosh*cosA*coslat*sinlon+n2*coslon/(fr*vimp)*(x*dxdlat+y*dydlat)-vimp*sinh*sinlat*sinlon,
            vimp*(-cosh*cosA*sinlat+sinh*coslat)
        ])

        #dX/dalt:
        Jalt=np.array([
            coslat*coslon,coslat*sinlon,sinlat,
            -n2*sinlon/(fr*vimp)*(x*coslat*coslon+y*coslat*sinlon),
            +n2*coslon/(fr*vimp)*(x*coslat*coslon+y*coslat*sinlon),
            0
        ])

        #dX/dA:
        JA=np.array([0,0,0,
            vimp*(cosh*sinA*sinlat*coslon-cosh*cosA*sinlon),
            vimp*(cosh*sinA*sinlat*sinlon+cosh*cosA*coslon),
            -vimp*cosh*sinA*coslat,
           ])

        #dX/dh:
        Jh=np.array([0,0,0,
            vimp*(sinh*cosA*sinlat*coslon+sinh*sinA*sinlon+cosh*coslat*coslon),
            vimp*(sinh*cosA*sinlat*sinlon-sinh*sinA*coslon+cosh*coslat*sinlon),
            vimp*(-sinh*cosA*coslat+cosh*sinlat),
            ])

        #dX/dvimp:
        Jv=np.array([0,0,0,vx/vimp+sinlon*fr,vy/vimp-coslon*fr,vz/vimp])

        self.Jcl=np.array([Jlon,Jlat,Jalt,JA,Jh,Jv]).transpose()
        self.Jlc=np.linalg.inv(self.Jcl)
    
    def propagateRay(self):
        
        state=np.zeros(6)

        body=self.body
        state=self.stateEcl+body.stateHelio
        et=self.tdb
        
        self.conics=[]
        while body.master is not None:

            #State w.r.t. to body
            body.updateBody(et)
            state=state-body.stateHelio
        
            #Get object-centric elements
            q,e,i,Omega,omega,Mo,et,mu=spy.oscelt(state,et,body.mu)
            a=q/(1-e)
            n=np.sqrt(body.mu/np.abs(a)**3)
            self.conics+=[[q,e,i,Omega,omega,Mo,body.mu]]
            
            #hill
            etp=et-Mo/n
            fd=np.arccos((q*(1+e)/body.rhill-1)/e)
            Hd=2*np.arctanh(np.sqrt((e-1)/(e+1))*np.tan(fd/2))
            Md=e*np.sinh(Hd)-Hd
            deltat=Md/n
            
            #Update body position
            body.updateBody(etp-deltat)

            #Heliocentric conic:
            hillstate=spy.conics([q,e,i,Omega,omega,Mo,et,body.mu],etp-deltat)
            self.conics+=[[q,e,i,Omega,omega,-Md,body.mu]]
            
            #Next conic
            et=etp-deltat
            state=hillstate+body.stateHelio

            body=self.masters[body.master]
        
        self.terminal=KeplerianOrbit(Spice.Mu["SSB"])
        self.terminal.setState(state,et)
        self.conics+=[list(self.terminal.elements)+[Spice.Mu["SSB"]]]
            
#Unitary test
class Test(unittest.TestCase):
    
    #Involved bodies
    earth=Body("EARTH")
    moon=Body("MOON")
    
    #Chelyabinsk impact
    #Time
    tdb_chely=Spice.str2t("02/15/2013 3:20:34 UTC")
    #Location
    chely=Location(earth,61.1*Angle.Deg,54.8*Angle.Deg,23.3*Const.km)
    #Ray
    ray_chely=GrtRay(chely,101.1*Angle.Deg,15.9*Angle.Deg,-18.6*Const.km)
    
    #Moon impact
    tdb_moon=Spice.str2t("2000 JAN 02 12:00:00 UTC")
    #Location
    crater=Location(moon,45.6452*Angle.Deg,41.1274*Angle.Deg,10.0*Const.km)
    #Ray
    ray_crater=GrtRay(crater,1.6502*Angle.Deg,56.981*Angle.Deg,-4.466*Const.km)
    
    #Arbitray impact
    tdb_arb=Spice.str2t("2000 JAN 01 12:00:00 UTC")
    site=Location(earth,25.0*Angle.Deg,53.0*Angle.Deg,100.0*Const.km)
    ray_site=GrtRay(site,40.0*Angle.Deg,20.0*Angle.Deg,-10.0*Const.km)
    
    #"""
    def test_ray_jacobian(self):
        self.ray_site.updateRay(self.tdb_arb)
        self.ray_site.calcJacobiansBody()

        self.assertEqual(
            np.isclose(self.ray_site.Jcl.flatten(),
                       [-1.65111080e+06,-4.68755962e+06,5.45429642e-01,0.00000000e+00,0.00000000e+0,0.00000000e+00,
                        3.54081854e+06,-2.18584495e+06,2.54338019e-01,0.00000000e+00,0.00000000e+00,0.00000000e+00,
                        0.00000000e+00,3.89749442e+06,7.98635510e-01,0.00000000e+00,0.00000000e+00,0.00000000e+00,
                        3.65693764e+03,6.56078885e+03,-1.84959827e-05,-1.32977275e+03,-7.95087289e+03,-5.89786728e-01,
                        5.77779511e+03,2.64433327e+03,3.96647629e-05,-8.56270845e+03,-1.28181727e+03,3.91443323e-01,
                        0.00000000e+00,3.69061965e+03,0.00000000e+00,3.63509979e+03,-5.92794777e+03,7.06345341e-01],
                       rtol=1e-5).tolist(),
                       [True]*36)
    
    def test_propagate_moon(self):
        self.timing_propagate_ray_moon()
        E=np.copy(self.ray_crater.terminal.elements)
        Const.transformElements(E,[1/Const.au,Angle.Rad],implicit=True)
        self.assertEqual(np.isclose(E,
                                    [9.28310700e-01,4.62115646e-02,6.82524233e+00,
                                     2.82253534e+02,2.81089088e+02,2.59457851e+02],
                                    rtol=1e-5).tolist(),
                         [True]*6)        

    def test_propagate_chely(self):
        self.timing_propagate_ray_earth()
        E=np.copy(self.ray_chely.terminal.elements)
        Const.transformElements(E,[1/Const.au,Angle.Rad],implicit=True)
        self.assertEqual(np.isclose(E,
                                    [0.73858102,0.54966922,4.04158232,
                                     326.57255475,106.86339209,21.32411541],
                                    rtol=1e-5).tolist(),
                         [True]*6)

    def test_state_chely(self):
        self.ray_chely.updateRay(self.tdb_chely)

        self.assertEqual(np.isclose(self.ray_chely.stateBody,
                                    [1.78729411e+06,3.23767769e+06,5.20762055e+06,
                                     1.23526608e+04,-1.33886204e+04,-2.17876404e+03],
                                    rtol=1e-5).tolist(),
                         [True]*6)
        self.assertEqual(np.isclose(self.ray_chely.stateEcl,
                                    [-8.82111395e+05,-1.22185261e+06,6.20687075e+06, 
                                     -1.53985024e+04,8.07537430e+03,-5.85361851e+03],
                                    rtol=1e-5).tolist(),
                         [True]*6)

    #"""
    
    def timing_update_ray(self):
        self.ray_chely.updateRay(self.tdb_chely)

    def timing_propagate_ray_earth(self):
        self.ray_chely.updateRay(self.tdb_chely)
        self.ray_chely.propagateRay()

    def timing_propagate_ray_moon(self):
        self.ray_crater.updateRay(self.tdb_moon)
        self.ray_crater.propagateRay()

if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    #"""
    print("Timing update ray:")
    %timeit -n 100 Test().timing_update_ray()
    
    print("Timing propagate ray earth:")
    %timeit -n 100 Test().timing_propagate_ray_earth()
    
    print("Timing propagate ray moon:")
    %timeit -n 100 Test().timing_propagate_ray_moon()
    #"""

....
----------------------------------------------------------------------
Ran 4 tests in 0.015s

OK


Timing update ray:
100 loops, best of 3: 234 µs per loop
Timing propagate ray earth:
100 loops, best of 3: 637 µs per loop
Timing propagate ray moon:
100 loops, best of 3: 926 µs per loop


In [311]:
from scipy.stats import multivariate_normal as multinorm
class MultiNormal(object):
    
    #Maximum range of locations and scales for bounds (intended for contraining minimization)
    MAXLS=10.0

    def pdf(self,r):
        value=0
        for w,loc,cov in zip(self.aweights,self.locs,self.covs):
            value+=w*multinorm.pdf(r,loc,cov)
        return value

    def rvs(self,N):
        """
        Generate a random sample of points following this MND
        """
        rs=[]
        for i in range(N):
            n=Util.genIndex(self.aweights)
            r=multinorm.rvs(self.locs[n],self.covs[n])
            rs+=[r]
        return rs

    def setUnflatten(self,weights,locs,scales,angles):

        #Store 
        self.weights=weights
        self.aweights=self.weights+[1-sum(self.weights)]
        self.locs=locs
        self.scales=scales
        self.angles=angles
        self.M=len(locs)
        self.params=sum([weights],[])+sum(locs,[])+sum(scales,[])+sum(angles,[])
        self.N=len(self.params)
        
        #Covariances
        self._calcCovariances()
        
        #Constraints for minimization
        self.calcBounds(self.MAXLS)
            
    def setFlatten(self,params):

        #Unflatten
        self.params=params
        N=len(self.params)
        if N>9:
            M=np.int((len(params)+1)/10)
            i=0;j=i+M-1
            weights=list(params[i:j])+[1-np.sum(params[i:j])]
        else:
            #Case for one function
            M=1
            weights=[1.0]
            j=0
        i=j;j=i+3*M
        self.locs=np.reshape(params[i:j],(M,3))
        i=j;j=i+3*M
        self.scales=np.reshape(params[i:j],(M,3))
        i=j;j=i+3*M
        self.angles=np.reshape(params[i:j],(M,3))

        self.N=N
        self.M=M
        
        #Covariances
        self._calcCovariances()

        #Constraints for minimization
        self.calcBounds(self.MAXLS)

    def calcBounds(self,maxls):
        M=self.M
        wbnds=(0,1),
        lbnds=(-maxls,maxls),
        sbnds=(1e-3,maxls),
        abnds=(-np.pi,np.pi),
        self.bounds=()
        if M>1:
            self.bounds=wbnds*(M-1)
        self.bounds+=lbnds*M*3+sbnds*M*3+abnds*M*3
        
    def _calcCovariances(self):
        rots=[]
        self.covs=[]
        for scale,angle in zip(self.scales,self.angles):
            L=np.identity(len(scale))*np.outer(np.ones(len(scale)),scale)
            spy.eul2m(-angle[0],-angle[1],-angle[2],3,1,3)
            rots+=[spy.eul2m(-angle[0],-angle[1],-angle[2],3,1,3)]
            self.covs+=[spy.mxm(spy.mxm(rots[-1],spy.mxm(L,L)),spy.invert(rots[-1]))]
    
#Unitary test
class Test(unittest.TestCase):

    mnd=MultiNormal()
    
    #"""
    def test_rvs(self):
        self.timing_set_unflatten()
        r=self.mnd.rvs(1)
        print(r)
    
    def test_pdf(self):
        self.timing_set_unflatten()
        p=self.mnd.pdf(self.mnd.locs[0])
        self.assertAlmostEqual(p,0.09989816167966402,7)
        
    
    def test_set_unflatten(self):
        self.timing_set_unflatten()
        self.assertEqual(self.mnd.M,2)
        self.assertEqual(self.mnd.N,10*self.mnd.M-1)
        
        self.assertEqual(np.isclose(self.mnd.aweights,
                                    [0.6,0.4],
                                    rtol=1e-5).tolist(),
                         [True]*2)
        self.assertEqual(np.isclose(self.mnd.params,
                                    [0.6, 
                                     0.5, 0.5, -2.0, 
                                     2.0, 0.3, -2.6, 
                                     1.3, 0.7, 0.5, 
                                     0.4, 0.9, 1.6, 
                                     -0.6981317007977318, -1.5009831567151235, 0.0, 
                                     1.3962634015954636, -1.9024088846738192, 0.0],
                                    rtol=1e-5).tolist(),
                         [True]*self.mnd.N)

        self.assertEqual(np.isclose(self.mnd.covs[0].flatten(),
                                    [1.09550921,-0.70848654,-0.01073505,
                                     -0.70848654,0.84565862,-0.01279353,
                                     -0.01073505,-0.01279353,0.48883217],
                                    rtol=1e-5).tolist(),
                         [True]*9)

        self.assertEqual(np.isclose(self.mnd.covs[1].flatten(),
                                    [2.30773378,-0.37870341,0.53051967,
                                     -0.37870341,0.22677563,-0.09354493,
                                     0.53051967,-0.09354493,0.99549059],
                                    rtol=1e-5).tolist(),
                         [True]*9)

    def test_set_flatten(self):
        self.timing_set_flatten()
        self.assertEqual(self.mnd.M,2)
        self.assertEqual(self.mnd.N,10*self.mnd.M-1)
        self.assertEqual(np.isclose(self.mnd.params,
                                    [0.6, 
                                     0.5, 0.5, -2.0, 
                                     2.0, 0.3, -2.6, 
                                     1.3, 0.7, 0.5, 
                                     0.4, 0.9, 1.6, 
                                     -0.6981317007977318, -1.5009831567151235, 0.0, 
                                     1.3962634015954636, -1.9024088846738192, 0.0],
                                    rtol=1e-5).tolist(),
                         [True]*self.mnd.N)

        self.assertEqual(np.isclose(self.mnd.covs[0].flatten(),
                                    [1.09550921,-0.70848654,-0.01073505,
                                     -0.70848654,0.84565862,-0.01279353,
                                     -0.01073505,-0.01279353,0.48883217],
                                    rtol=1e-5).tolist(),
                         [True]*9)

        self.assertEqual(np.isclose(self.mnd.covs[1].flatten(),
                                    [2.30773378,-0.37870341,0.53051967,
                                     -0.37870341,0.22677563,-0.09354493,
                                     0.53051967,-0.09354493,0.99549059],
                                    rtol=1e-5).tolist(),
                         [True]*9)

    #""" 
    
    def timing_set_unflatten(self):
        weights=[0.6]
        locs=[
            [0.5,0.5,-2.0],
            [2.0,0.3,-2.6]
        ]
        scales=[
            [1.3,0.7,0.5],
            [0.4,0.9,1.6]
        ]
        angles=[
            [-40.0*Angle.Deg,-86.0*Angle.Deg,0.0*Angle.Deg],
            [+80.0*Angle.Deg,-109.0*Angle.Deg,0.0*Angle.Deg]
        ]
        self.mnd.setUnflatten(weights,locs,scales,angles)
        
    def timing_set_flatten(self):
        params=[
            #weights
            0.6,
            #locs
            0.5, 0.5, -2.0,
            2.0, 0.3, -2.6, 
            #scales
            1.3, 0.7, 0.5,
            0.4, 0.9, 1.6, 
            #Angles
            -40.0*Angle.Deg,-86.0*Angle.Deg,0.0*Angle.Deg,
            +80.0*Angle.Deg,-109.0*Angle.Deg,0.0*Angle.Deg
        ]
        self.mnd.setFlatten(params)

if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    #"""
    print("Timing set unflatten:")
    %timeit -n 100 Test().timing_set_unflatten()

    print("Timing set flatten:")
    %timeit -n 100 Test().timing_set_flatten()
    
    print("Timing PDF:")
    t=Test()
    %timeit -n 100 t.mnd.pdf([0,0,0])

    print("Timing RVS:")
    t=Test()
    %timeit -n 100 t.mnd.rvs(1)
    #"""
    
    

....
----------------------------------------------------------------------
Ran 4 tests in 0.016s

OK


[array([ 1.52940205,  0.58595752, -2.80589524])]
Timing set unflatten:
100 loops, best of 3: 420 µs per loop
Timing set flatten:
100 loops, best of 3: 586 µs per loop
Timing PDF:
100 loops, best of 3: 363 µs per loop
Timing RVS:
100 loops, best of 3: 298 µs per loop


In [11]:
class Template(object):
    
    def method(self):
        pass
    
#Unitary test
class Test(unittest.TestCase):

    #"""
    def test(self):
        self.timing()
    #""" 
    
    def timing(self):
        pass    

if __name__=='__main__':
    #Testing
    unittest.main(argv=['first-arg-is-ignored'],exit=False)
    
    #Timing
    #"""
    print("Timing:")
    %timeit -n 1000 Test().timing()
    #"""

.

Timing:
1000 loops, best of 3: 4.18 µs per loop



----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
